In [1]:
import keras
from keras import layers
from keras import ops

2025-11-17 11:22:49.043674: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class Antirectifier(layers.Layer):
    def __init__(self, initializer="he_normal", **kwargs):
        super().__init__(**kwargs)
        self.initializer = keras.initializers.get(initializer)

    def build(self, input_shape):
        output_dim = input_shape[-1]
        self.kernel = self.add_weight(
            shape=(output_dim * 2, output_dim),
            initializer=self.initializer,
            name="kernel",
            trainable=True,
        )

    def call(self, inputs):
        inputs -= ops.mean(inputs, axis=-1, keepdims=True)
        pos = ops.relu(inputs)
        neg = ops.relu(-inputs)
        concatenated = ops.concatenate([pos, neg], axis=-1)
        mixed = ops.matmul(concatenated, self.kernel)
        return mixed

    def get_config(self):
        # Implement get_config to enable serialization. This is optional.
        base_config = super().get_config()
        config = {"initializer": keras.initializers.serialize(self.initializer)}
        return dict(list(base_config.items()) + list(config.items()))

In [3]:
# Training parameters
batch_size = 128
num_classes = 10
epochs = 20

# The data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# Build the model
model = keras.Sequential(
    [
        keras.Input(shape=(784,)),
        layers.Dense(256),
        Antirectifier(),
        layers.Dense(256),
        Antirectifier(),
        layers.Dropout(0.5),
        layers.Dense(10),
    ]
)

# Compile the model
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.15)

# Test the model
model.evaluate(x_test, y_test)

60000 train samples
10000 test samples


I0000 00:00:1763374971.178843   74332 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9059 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:08:00.0, compute capability: 7.5


Epoch 1/20


2025-11-17 11:22:52.376151: I external/local_xla/xla/service/service.cc:163] XLA service 0x7a81b8003ce0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-17 11:22:52.376161: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2025-11-17 11:22:52.394350: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-17 11:22:52.473334: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91600


151/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9395 - sparse_categorical_accuracy: 0.7151

I0000 00:00:1763374973.194784   74425 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


399/399 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.3762 - sparse_categorical_accuracy: 0.8883 - val_loss: 0.1971 - val_sparse_categorical_accuracy: 0.9378
Epoch 2/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1742 - sparse_categorical_accuracy: 0.9516 - val_loss: 0.4285 - val_sparse_categorical_accuracy: 0.8937
Epoch 3/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1389 - sparse_categorical_accuracy: 0.9614 - val_loss: 0.1974 - val_sparse_categorical_accuracy: 0.9489
Epoch 4/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1134 - sparse_categorical_accuracy: 0.9688 - val_loss: 0.1482 - val_sparse_categorical_accuracy: 0.9657
Epoch 5/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1000 - sparse_categorical_accuracy: 0.9734 - val_loss: 0.1279 - val_sparse_categorical_accuracy: 0.9716
Epoch 6/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0857 - sparse_categorical_accuracy: 0.9773 - val_loss: 0.1132 - val_sparse_categorical_accuracy: 0.9747
Epoch 7/20
399/

[0.3570076823234558, 0.9617000222206116]